In [1]:
# Dependencies and Setup (This section of data was given to us)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from google_key import g_key

In [2]:
# (Importing data from Part I into a Dataframe)
weather_df = pd.read_csv("../WeatherPy/Output/cities.csv")
weather_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lethem,3.3803,-59.7968,78.91,61,38,13.00,GY,1645154267
1,Antalaha,-14.9003,50.2788,74.61,88,85,6.78,MG,1645154267
2,Rikitea,-23.1203,-134.9692,78.31,71,7,7.70,PF,1645154267
3,Rome,43.2128,-75.4557,54.66,97,100,7.00,US,1645154200
4,Yellowknife,62.4560,-114.3525,-12.59,85,100,13.00,CA,1645154091
...,...,...,...,...,...,...,...,...,...
567,Bridgetown,13.1000,-59.6167,75.69,83,40,16.11,BB,1645155169
568,Provincia di Imperia,43.9667,7.7833,50.00,87,0,5.08,IT,1645155170
569,Chicama,-7.8447,-79.1469,72.14,75,98,4.50,PE,1645155170
570,Krasne,49.9145,24.6134,38.55,79,99,18.52,UA,1645155170


In [3]:
# (Creating the heatmap itself)
gmaps.configure(api_key=g_key)

locations = weather_df[["Latitude", "Longitude"]] # (Storing the values to be used later)
humidity = weather_df["Humidity"]

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
max_intensity # (To double-check that the highest value is not above 100%)

100

In [5]:
# (Narrowing down Dataframe data to ideal weather conditions provided by the assignment)
# This includes:
# - Max Temperature lower than 80 but higher than 70
# - Wind speed less than 10 mph
# - Zero cloudiness

narrowed_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) &  
                             (weather_df["Max Temperature"] >= 70) & (weather_df["Max Temperature"] <= 80)].dropna()
narrowed_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
84,Lázaro Cárdenas,17.9583,-102.2000,74.64,84,0,1.10,MX,1645154361
131,Coahuayana Viejo,18.7333,-103.6833,70.65,87,0,1.83,MX,1645154440
238,Cooma,-36.2333,149.1333,76.77,33,0,9.22,AU,1645154604
242,Sur,22.5667,59.5289,70.79,57,0,9.28,OM,1645154455
251,Niamey,13.5137,2.1098,75.49,20,0,3.44,NE,1645154609
269,San Juan Bautista Tuxtla,18.1000,-96.1167,72.23,83,0,0.76,MX,1645154599
286,Pochutla,15.7432,-96.4661,79.65,77,0,3.83,MX,1645154683
293,Sarankhola,22.3082,89.7897,73.24,34,0,6.98,BD,1645154686
309,Santiago,-33.4569,-70.6483,73.85,41,0,8.05,CL,1645154531
349,Vila Velha,-20.3297,-40.2925,76.95,86,0,5.75,BR,1645154770


In [6]:
# (Storing data into a variable called "hotel_df" and adding a column that will store the names of the hotels that are
#  nearby the cities)
hotel_df = narrowed_df.loc[:,["City", "Latitude", "Longitude", "Country"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Latitude,Longitude,Country,Hotel Name
84,Lázaro Cárdenas,17.9583,-102.2000,MX,
131,Coahuayana Viejo,18.7333,-103.6833,MX,
238,Cooma,-36.2333,149.1333,AU,
242,Sur,22.5667,59.5289,OM,
251,Niamey,13.5137,2.1098,NE,
269,San Juan Bautista Tuxtla,18.1000,-96.1167,MX,
286,Pochutla,15.7432,-96.4661,MX,
293,Sarankhola,22.3082,89.7897,BD,
309,Santiago,-33.4569,-70.6483,CL,
349,Vila Velha,-20.3297,-40.2925,BR,


In [7]:
# (Gathering data using Google Places to find the nearest hotels in a 5000 mile radius to the associated city)
# [Note: the format of the printing was inspired by an outside source]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
parameters = {"type": "hotel", "keyword": "hotel", "radius": 5000, "key": g_key}

print("-------Beginning Search-------")
print("------------")

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city_name = row["City"]
    parameters["location"] = f"{latitude},{longitude}"
    print(f"Retrieving results for Index {index}: {city_name}...")
    response = requests.get(base_url, params=parameters).json()
    results = response["results"]
    try:
        print(f"Closest hotel near/in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except(KeyError, IndexError):
        print("Missing data. Skipping...")
    print("------------")

print("-------End of Search-------")

-------Beginning Search-------
------------
Retrieving results for Index 84: Lázaro Cárdenas...
Closest hotel near/in Lázaro Cárdenas is Baymont by Wyndham Lazaro Cardenas.
------------
Retrieving results for Index 131: Coahuayana Viejo...
Closest hotel near/in Coahuayana Viejo is Hotel Aeropuerto.
------------
Retrieving results for Index 238: Cooma...
Closest hotel near/in Cooma is Alpine Hotel.
------------
Retrieving results for Index 242: Sur...
Closest hotel near/in Sur is Sur Plaza Hotel.
------------
Retrieving results for Index 251: Niamey...
Closest hotel near/in Niamey is Bravia Hotel Niamey.
------------
Retrieving results for Index 269: San Juan Bautista Tuxtla...
Closest hotel near/in San Juan Bautista Tuxtla is Hotel Meson.
------------
Retrieving results for Index 286: Pochutla...
Closest hotel near/in Pochutla is Hotel San Pedro.
------------
Retrieving results for Index 293: Sarankhola...
Missing data. Skipping...
------------
Retrieving results for Index 309: Santiag

In [8]:
hotel_df # (Showing the Dataframe to confirm that the hotel data has been added)

,City,Latitude,Longitude,Country,Hotel Name
84,Lázaro Cárdenas,17.9583,-102.2000,MX,Baymont by Wyndham Lazaro Cardenas
131,Coahuayana Viejo,18.7333,-103.6833,MX,Hotel Aeropuerto
238,Cooma,-36.2333,149.1333,AU,Alpine Hotel
242,Sur,22.5667,59.5289,OM,Sur Plaza Hotel
251,Niamey,13.5137,2.1098,NE,Bravia Hotel Niamey
269,San Juan Bautista Tuxtla,18.1000,-96.1167,MX,Hotel Meson
286,Pochutla,15.7432,-96.4661,MX,Hotel San Pedro
293,Sarankhola,22.3082,89.7897,BD,
309,Santiago,-33.4569,-70.6483,CL,Sheraton Santiago Hotel and Convention Center
349,Vila Velha,-20.3297,-40.2925,BR,Hotel Golden Tulip Porto Vitória


In [9]:
# NOTE: Do not change any of the code in this cell (This was given to us)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# (Creating the locations to be shown on the heatmap created previously)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))